[`DetectionValidator`](https://github.com/ultralytics/ultralytics/blob/main/ultralytics/models/yolo/detect/val.py)

In [1]:
%matplotlib inline
__import__("os").environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

In [2]:
import os
import copy
import matplotlib.pyplot as plt

import torch
import random
import numpy as np

seed=0
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [3]:
import sys
sys.path.append('../../..')

%load_ext autoreload
%autoreload 2

from computer_vision.yolov11.utils.check import check_imgsz
from computer_vision.yolov11.modules.detector import DetectionModel
from computer_vision.yolov11.parameter_parser import parser
from computer_vision.yolov11.data.dataset import YOLODataset
from computer_vision.yolov11.modules.validator import DetectionValidator

In [4]:
data_dirpath=r'D:/data/ultralytics/coco128'
result_dirpath='D:/results/yolov11/training'

argument=f'''--root {data_dirpath} --train-image-dirname images/train2017 --train-label-dirname labels/train2017
--val-image-dirname images/train2017 --val-label-dirname labels/train2017
--data-cfg ../coco128.yaml --hyperparam ../default.yaml --model-cfg ../yolo11.yaml 
--batch-size 2 --output-dirpath {result_dirpath} --checkpoint-dirpath {result_dirpath}/checkpoints 
--save-period 1 --save-json '''
args=parser.parse_args(argument.split())

if not os.path.isdir(args.checkpoint_dirpath): os.makedirs(args.checkpoint_dirpath)

[`DetMetrics`](https://github.com/ultralytics/ultralytics/blob/main/ultralytics/utils/metrics.py#L1032)

In [5]:
from __future__ import annotations

import os
import yaml
import copy
from pathlib import Path
from typing import Any
from argparse import Namespace

import torch
import numpy as np
from computer_vision.yolov11.utils.metrics import Metric
from computer_vision.yolov11.utils import ops

[`DetectionValidator`](https://github.com/ultralytics/ultralytics/blob/main/ultralytics/models/yolo/detect/val.py#L16)

In [6]:
model_dirpath='D:/results/yolov11/checkpoints'
# to load ultralytics models (models downloaded previously by ultralytics) we need to use ultralytics library, as a work around,
# we resave it using torch to remove ultralytics dependency
# checkpoint=torch.load(os.path.join(model_dirpath, 'yolo11n.pt'), map_location=torch.device('cpu'), weights_only=False)

model=DetectionModel(cfg='../yolo11.yaml', ch=3)
checkpoint=torch.load(os.path.join(model_dirpath, 'predictor_yolo11n_torch.pt'), map_location=torch.device('cpu'), weights_only=True) # may have model. as added prefix
try:
    model.load_state_dict(checkpoint['model'])
except RuntimeError as err:
    state_dict=copy.deepcopy(model.state_dict())
    for name, params in checkpoint['model'].items():
        name=name[len('model.'):] # each parameter name is model.model.xxx so we need to remove 1 model.
        
        if name not in state_dict or params.shape!=state_dict[name].shape:
            print(name, name in state_dict, params.shape!=state_dict[name].shape if name in state_dict else None)
        else: state_dict[name]=params
    model.load_state_dict(state_dict)
    print('Load model with modify name')

Load model with modify name


In [7]:
val_dataset=YOLODataset(img_path=(Path(args.root)/args.val_image_dirname),
                        label_path=(Path(args.root)/args.val_label_dirname),
                        data=args.data_cfg, hyp=args.hyperparam, imgsz=args.imgsz, cache=True, augment=False, rect=False, 
                        batch_size=args.batch_size, stride=32, pad=0.5,  single_cls=False, classes=None, fraction=1., channels=3)
val_loader=torch.utils.data.DataLoader(dataset=val_dataset, batch_size=args.batch_size, shuffle=False, sampler=None, batch_sampler=None, 
                                       num_workers=0, collate_fn=YOLODataset.collate_fn, pin_memory=False, drop_last=False, 
                                       timeout=0, worker_init_fn=None, prefetch_factor=None, persistent_workers=False)
validator=DetectionValidator(hyperparam=Path(args.hyperparam), data_cfg=val_dataset.data, dataloader=val_loader, 
                 save_dir=Path(args.output_dirpath), args=args)

In data.dataset.YOLODataset.update_images_labels cache path D:\data\ultralytics\coco128\labels\train2017.cache exist. Load it!!!
Scanning D:\data\ultralytics\coco128\labels\train2017.cache ... 126 images with 2 missing and 0 empty files as well as 0 corrupt files
max_buffer_length  0  ni  128


In [9]:
validator(model=model)

                   all        128        929      0.742       0.56      0.679      0.513
                     0         61        254      0.887      0.616      0.788      0.558
                     1          3          6      0.347      0.167      0.349      0.198
                     2         12         46      0.769      0.174      0.253      0.176
                     3          4          5      0.784          1      0.962      0.785
                     4          5          6          1      0.945      0.995      0.929
                     5          5          7       0.81      0.714      0.741      0.694
                     6          3          3      0.866          1      0.995       0.91
                     7          5         12      0.435      0.167       0.41      0.246
                     8          2          6      0.698      0.397      0.518      0.389
                     9          4         14      0.551      0.143      0.254      0.175
                    1

{'metrics/precision(B)': 0.741509716636013,
 'metrics/recall(B)': 0.5597794926183444,
 'metrics/mAP50(B)': 0.6792360308149096,
 'metrics/mAP50-95(B)': 0.5131817304679367,
 'fitness': 0.5131817304679367}